# Hyperparameter Optimization For Ridge Regression Model

In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
from functools import *

import sys
sys.path.append("../")
from ddn.node import *

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:

class RidgeNode(AbstractDeclarativeNode):
    
    def __init__(self, n):
        super().__init__(1, n)
        
    # x is miu
    # y is [beta_1^T,beta_2^T,...,beta_K^T]
    def objective(self, x, y):
        sum = 0
        for i in range(K):
            sum += 0.5 * np.sum(np.square(np.dot(A_train_splitted[i],y[i])-b_train_splitted[i])) + 0.5 * x * np.linalg.norm(y[i],ord = 2)
        return sum

    def solve(self, x):
        # TODO
        result = np.array([])
        for i in range(K):
            if i == 0:
                result = sci.linalg.solve(np.dot(A_train_splitted[i].T,A_train_splitted[i]) + x * I_out, np.dot(A_train_splitted[i].T, b_train_splitted[i]))
            else:
                result = np.vstack((result, sci.linalg.solve(np.dot(A_train_splitted[i].T,A_train_splitted[i]) + x * I_out, np.dot(A_train_splitted[i].T,b_train_splitted[i]))))
            # print(result.shape)
        return result, None

    def gradient(self, x, y=None, ctx=None):
        # TODO
        if y is None:
            y, ctx = self.solve(x)
        
        result = np.array([])
        for i in range(K):
            if i == 0:
                result = (-1) * sci.linalg.solve(np.dot(A_train_splitted[i].T,A_train_splitted[i]) + x * I_out, y[i])
            else:
                result = np.vstack((result, (-1) * sci.linalg.solve(np.dot(A_train_splitted[i].T,A_train_splitted[i]) + x * I_out, y[i])))
            # print(result.shape)
        
        return result

In [ ]:
def function_objective(y, A_test_splitted, b_test_splitted):
    sum = 0
    for i in range(K):
        y_k = y[i]
        sum += (0.5) * np.sum(np.square(np.dot(A_test_splitted[i], y_k) - b_test_splitted[i]))
    return sum


def function_objective_one(y, A_test_splitted, b_test_splitted):
    return (0.5) * np.sum(np.square(np.dot(A_test_splitted, y) - b_test_splitted))


def derivative_objective_y(y, A_test_splitted, b_test_splitted):
    result = np.array([])
    for i in range(K):
        if i == 0:
            result = np.dot((np.dot(y[i].T, A_test_splitted[i].T) - b_test_splitted[i].T), A_test_splitted[i])
        else:
            result = np.vstack((result, np.dot((np.dot(y[i].T, A_test_splitted[i].T) - b_test_splitted[i].T), A_test_splitted[i])))
        # print(result.shape)
    return result


In [ ]:
def simpleGradientDescent(node, miu_init, b_test_splitted, A_test_splitted, step_size=1.0e-1, tol=1.0e-8, max_iters=1500000, verbose=False):
    # Initialize
    cnt = 0
    miu = miu_init
    all_miu = []
    gradient = []
    axis_x = []
    history = []
    loss = []

    # Iteration
    for i in range(max_iters):
        # solve the lower-level problem and compute the upper-level objective
        y, _ = node.solve(miu)
        history.append(function_objective(y, A_test_splitted, b_test_splitted))
        if verbose: print("{:5d}: {}".format(i, history[-1]))
        if (len(history) > 2) and (history[-2] - history[-1]) < tol:
            print(y)
            print(function_objective(y, A_test_splitted, b_test_splitted))
            break
        
        # print(derivative_objective_y(y, A_test_splitted, b_test_splitted).shape)
        # compute the gradient of the upper-level objective with respect to x via the chain rule
        dJdx = 0 
        for j in range(K):
            # print(derivative_objective_y(y, A_test_splitted, b_test_splitted)[j].shape)
            # print(node.gradient(miu, y)[j].shape)
            dJdx += np.dot((derivative_objective_y(y, A_test_splitted, b_test_splitted)[j]).T, node.gradient(miu, y)[j])

        if i == 0:
            print(node.gradient(miu, y))
            print(node.gradient(miu, y).reshape((out*K,1)))
            print(y)
            
        # take a step in the negative gradient direction
        miu -= step_size * dJdx
        all_miu.append(miu)
        gradient.append(abs(dJdx))
        cnt += 1
        axis_x.append(cnt)
        loss.append(function_objective(y, A_test_splitted, b_test_splitted) / (K * A_test_splitted[0].shape[0]))

    plt.figure(1)
    plt.plot(axis_x, all_miu)
    plt.xlabel("iteration time")
    plt.ylabel("hyperparameter mu")

    plt.figure(2)
    plt.plot(all_miu, gradient)
    plt.xlabel("hyperparameter mu")
    plt.ylabel("gradient")
    print(gradient[-1])

    plt.figure(3)
    plt.plot(all_miu, loss)
    plt.xlabel("hyperparameter mu")
    plt.ylabel("loss function")
    print(loss[-1])

    print("cnt: ", cnt)
    return miu, history

In [ ]:
# Download DataSet
iris = datasets.load_iris()
A = iris.data  
b = iris.target  

scaler = StandardScaler()
A_scaled = scaler.fit_transform(A)

# K-fold
K = 5
KF = KFold(n_splits = K)

A_train_splitted = []
A_test_splitted = []
b_train_splitted = []
b_test_splitted = []

for train_index, test_index in KF.split(A_scaled):
    A_train, A_test = A_scaled[train_index], A_scaled[test_index]
    A_train_splitted.append(A_train)
    A_test_splitted.append(A_test)
    b_train, b_test = b[train_index], b[test_index]
    b_train_splitted.append(b_train)
    b_test_splitted.append(b_test)
    print("X_train: ", A_train.shape)
    print("X_test: ", A_test.shape)
    
out = A_train_splitted[0].shape[1]
I_out = np.eye(out)

node = RidgeNode(K*out)
miu_init = 800
x, history_gd = simpleGradientDescent(node, miu_init, b_test_splitted, A_test_splitted)
y_star, _ = node.solve(x)
print("Gradient descent to give x = {}".format(x))
# Visualization
plt.show()